In [1]:
%pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 30.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

In [65]:
posts_number = 120

In [67]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.facebook.com")
time.sleep(3)

with open("cookies.json", "r") as file:
    cookies = json.load(file)
    for cookie in cookies:
        driver.add_cookie(cookie)

driver.refresh()
time.sleep(5)

page_url = "https://www.facebook.com/providentrealestateuz"
driver.get(page_url)
time.sleep(5)

posts = driver.find_elements(By.CSS_SELECTOR, "div[data-pagelet^='TimelineFeedUnit']")

while (len(posts) < posts_number):
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(5)
    posts = driver.find_elements(By.XPATH, "//div[contains(@data-pagelet,'TimelineFeedUnit')]")

with open("facebook_posts.jsonl", "w", encoding="utf-8") as jsonl_file:
    for post in posts:
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", post)
            WebDriverWait(post, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span>a[href*='/posts/']")))
        
            post_url_element = post.find_element(By.CSS_SELECTOR, "span>a[href*='/posts/']")
            post_url = post_url_element.get_attribute("href") if post_url_element else "No URL"
        # print(f"URL: {post_url}\n")

            WebDriverWait(post, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span>a[href*='/posts/']")))
            post_date_element = post.find_element(By.CSS_SELECTOR, "span>a[href*='/posts/']")
            post_date = post_date_element.text if post_date_element else "No Date"
        # print(f"Date: {post_date}\n")

            more_button = WebDriverWait(post, 20).until(EC.element_to_be_clickable(post.find_element(By.CSS_SELECTOR, "div[dir='auto'] [role='button']")))
            driver.execute_script("arguments[0].click();", more_button)
            WebDriverWait(post, 20).until(EC.presence_of_element_located((By.XPATH, "//*[@data-ad-comet-preview='message']")))

            sub_elements = post.find_elements(By.XPATH, ".//*[@data-ad-comet-preview='message']/div/div/span/div/div")
            all_text = ""
            for elem in sub_elements:
                all_text += elem.text + "\n"

            post_data = {
                "url": post_url,
                "date": post_date,
                "description": all_text
            }

            
            jsonl_file.write(json.dumps(post_data, ensure_ascii=False) + "\n")
        except:
            pass

        # print(f"Description: {all_text}")
        # print("-" * 50)

driver.quit()
